In [10]:
import pandas as pd
import numpy as np
import math, json, sys, io

In [11]:
example_data = [{'domain':'www.google.com'},{'domain':'outlook.office.com'},{'domain':'fjklbhwqejfb.plufty.io'},{'domain':'www.cnn.com'}]
df = pd.DataFrame(example_data)

In [12]:
import math
from collections import Counter
 
def getEntropy(data):
    length = float(len(data))
    return -sum(map(lambda x: (x/length)*math.log2(x/length), Counter(data).values()))

In [21]:
import pandas as pd
from pandas.io.json import json_normalize
from sklearn.model_selection import train_test_split
from sklearn import svm, metrics
import math 
from collections import Counter
import tldextract
import json
from flare.tools.alexa import Alexa

df = pd.DataFrame(example_data)
df.head()
 
#df = get_data_from_elasticsearch()
#df = df.rename(columns={'_source.Query':'domain'})
#df.head()
 
df['tld'] = df['domain'].apply(tldextract.extract)
df['tld'] = df.apply(lambda x: tldextract.extract(x['domain']).registered_domain,axis=1)
df['subdomain'] = df.apply(lambda x: tldextract.extract(x['domain']).subdomain,axis=1)
df['domain_entropy']=df['domain'].apply(getEntropy)
df['subdomain_length']=df['subdomain'].apply(len)
#df=df.drop(['_id','_index','_type'],axis=1)
df.head()
 
model=svm.OneClassSVM(kernel='linear', gamma=0.001, nu=0.95)
model=svm.OneClassSVM(kernel='rbf', gamma=0.00005)
data = df[['domain_entropy','subdomain_length']]
#print(data.head())
 
model.fit(df[['domain_entropy','subdomain_length']])
preds = model.predict(df[['domain_entropy','subdomain_length']])
anomalies = df[model.predict(df[['domain_entropy','subdomain_length']]) == 1]
#anomalies.shape()
#print(preds)
 
alexa=Alexa()
anomalies['in_alexa'] = anomalies['tld'].apply(alexa.domain_in_alexa)
print(anomalies.loc[anomalies['in_alexa'] == False])


                   domain        tld     subdomain  domain_entropy  \
2  fjklbhwqejfb.plufty.io  plufty.io  fjklbhwqejfb        3.879664   

   subdomain_length  in_alexa  
2                12     False  


/usr/lib/python3/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
